# Import Library

In [8]:
import pandas as pd

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk import pos_tag
from nltk.tag import pos_tag

import newspaper
from newspaper import Article

import string
import json
import operator
from collections import Counter

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction import DictVectorizer

import re
import time

# Read Data

In [3]:
header = ["GlobalEventId", "sqldate", "MonthYear","Year",
               "Actor1Code","Actor1Name","Actor1CountryCode","Actor1KnownGroupCode","Actor1Type1Code",
               "Actor2Code","Actor2Name","Actor2CountryCode","Actor2KnownGroupCode","Actor2Type1Code",
               "IsRootEvent","EventCode","EventBaseCode","EventRootCode",
               "QuadClass","GoldsteinScale","NumMentions","NumSources","NumArticles","AvgTone",
               "Actor1Geo_FullName","Actor1Geo_CountryCode","Actor1Geo_ADM1Code","Actor1Geo_Lat","Actor1Geo_Long",
               "Actor2Geo_FullName","Actor2Geo_CountryCode","Actor2Geo_ADM1Code","Actor2Geo_Lat","Actor2Geo_Long",
               "ActionGeo_FullName","ActionGeo_CountryCode","ActionGeo_ADM1Code","ActionGeo_Lat","ActionGeo_Long",
               "SOURCEURL"]

In [4]:
#ubah path_clean sesuai tempat clean di lokal
path_clean = 'E:/digitalent/2016-clean.csv'
#path_clean = 'E:/digitalent-project/csv/2017-clean.csv'
data_gdelt = pd.read_csv(path_clean, delimiter = ',', encoding = "ISO-8859-1", names=header)
data_gdelt.sample(4)

,GlobalEventId,sqldate,MonthYear,Year,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1Type1Code,Actor2Code,...,Actor2Geo_CountryCode,Actor2Geo_ADM1Code,Actor2Geo_Lat,Actor2Geo_Long,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,SOURCEURL
1144227,591789438,20161023,201610,2016,ITA,ITALY,ITA,NaN,NaN,BUS,...,IT,IT09,45.4667,9.2,"Milan, Lombardia, Italy",IT,IT09,45.4667,9.2000,http://energy.einnews.com/article/350625569/live
1012115,580546591,20160918,201609,2016,CHR,CHRISTIAN,NaN,NaN,NaN,CVL,...,GM,GM16,52.5167,13.4,"Berlin, Berlin, Germany",GM,GM16,52.5167,13.4000,http://news.webindia123.com/news/Articles/Worl...
1199592,596671889,20161107,201611,2016,MED,WRITER,NaN,NaN,MED,NaN,...,NaN,NaN,NaN,NaN,"Massachusetts, United States",US,USMA,42.2373,-71.5314,http://www.jta.org/2016/11/07/news-opinion/pol...
1251725,601872165,20161123,201611,2016,GHA,GHANA,GHA,NaN,NaN,GHAGOV,...,SY,SY,35.0000,38.0,Syria,SY,SY,35.0000,38.0000,http://www.myjoyonline.com/politics/2016/Novem...


### Inisialisasi Stopwords

In [5]:
stop_words = set(stopwords.words('english'))

### Inisialisasi Stemming

In [6]:
ps = nltk.stem.PorterStemmer()

### Inisialisasi dict res untuk menampung result

In [19]:
res = {}

# Fungsi Scrapping

In [21]:
# sebelum file json ada
def scrap_article(gid, source):
    start = time.time()
    article = Article(source)
    article.download()
    article.parse()
    article.nlp()
    
    #LOWERCASE
    txt = article.text.lower()
    
    #change nonword to space   
    txt_clean = re.sub(r'[^a-zA-Z]', ' ', txt)
    
    #TOKENISASI , STEM, HAPUS PROPERNOUN, DAN HAPUS STOPWORD
    tagged_sentence = nltk.pos_tag(txt_clean.split())
    
    filtered_sentence=[]
    for initword, tag in tagged_sentence:
        word = ps.stem(initword)
        if len(word)>1 and (word not in stop_words) and tag != 'NNP' and tag != 'NNPS':
            filtered_sentence.append(word)

    #print("TOKENS",type(filtered_sentence))
    
    #COUNT VECTOR: hitung kemunculan tiap kata
    gid_str = str(gid)
    val = dict((Counter(filtered_sentence)))
    
    #masukkan hasil ke dict res dengan id : gid_str dan value: val
    res[gid_str] = val
    end = time.time()
    waktu = end-start
    #print(waktu)
    return val
    

## WARNING!!

In [ ]:
#jalankan jika sudah ada file json
def scrap_article(gid, source):
    start = time.time()
    article = Article(source)
    article.download()
    article.parse()
    article.nlp()
    
    #LOWERCASE
    txt = article.text.lower()
    
    #change nonword to space   
    txt_clean = re.sub(r'[^a-zA-Z]', ' ', txt)
    
    #TOKENISASI , STEM, HAPUS PROPERNOUN, DAN HAPUS STOPWORD
    tagged_sentence = nltk.pos_tag(txt_clean.split())
    
    filtered_sentence=[]
    for initword, tag in tagged_sentence:
        word = ps.stem(initword)
        if len(word)>1 and (word not in stop_words) and tag != 'NNP' and tag != 'NNPS':
            filtered_sentence.append(word)

    #print("TOKENS",type(filtered_sentence))
    
    #COUNT VECTOR: hitung kemunculan tiap kata
    gid_str = str(gid)
    val = dict((Counter(filtered_sentence)))
    
    #masukkan hasil ke dict res dengan id : gid_str dan value: val
    res[gid_str] = val
    
    #update JSON file
    a_dict = {gid_str: val}

    with open('data.json') as f:
        data = json.load(f)

    data.update(a_dict)

    with open('data.json', 'w') as f:
        json.dump(data, f)
    
    end = time.time()
    waktu = end-start
    #print(waktu)
    return val

##### Contoh 1 url

In [18]:
sample=data_gdelt['SOURCEURL'][0]
sample
sample_ID = data_gdelt['GlobalEventId'][0]
scrap_article(sample_ID, sample)

{'close': 1,
 'rochest': 6,
 'man': 2,
 'accus': 2,
 'isi': 10,
 'sympath': 2,
 'plan': 7,
 'kill': 7,
 'patron': 3,
 'restaur': 3,
 'bar': 3,
 'new': 10,
 'year': 13,
 'eve': 8,
 'feder': 6,
 'author': 6,
 'alleg': 3,
 'angi': 1,
 'nassar': 1,
 'gari': 1,
 'craig': 1,
 'emanuel': 4,
 'lutchman': 26,
 'photo': 1,
 'provid': 3,
 'attack': 5,
 'say': 5,
 'could': 3,
 'gruesom': 1,
 'assault': 2,
 'islam': 4,
 'state': 9,
 'arm': 1,
 'knive': 2,
 'machet': 2,
 'take': 1,
 'life': 1,
 'old': 1,
 'allegedli': 6,
 'said': 11,
 'tuesday': 1,
 'hi': 9,
 'appar': 1,
 'plot': 4,
 'merchant': 3,
 'grill': 2,
 'road': 1,
 'problem': 1,
 'fbi': 12,
 'joint': 2,
 'terror': 3,
 'task': 2,
 'forc': 2,
 'wednesday': 2,
 'arrest': 8,
 'charg': 3,
 'offer': 1,
 'materi': 3,
 'support': 2,
 'terrorist': 5,
 'appear': 3,
 'court': 6,
 'thursday': 10,
 'morn': 1,
 'media': 2,
 'wa': 10,
 'alert': 1,
 'advanc': 1,
 'schedul': 3,
 'return': 2,
 'jan': 1,
 'detain': 1,
 'populac': 1,
 'alreadi': 1,
 'fear': 1,

In [13]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\suci.lestari\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# Proses Menghasilkan Dict

In [22]:
start2=time.time()
progress = 0
lenart = 5
for x in range(5):
    global start
    global progress
    progress += 1
    try:
        sample_=data_gdelt['SOURCEURL'][x]
        sample_ID_ = data_gdelt['GlobalEventId'][x]
        aaa = scrap_article(sample_ID_, sample_)
        #print(aaa, type(aaa))
    except:
        print('gagal', sample_)
        pass
    end = time.time()
    print('\r {} percent, {} position, {} per second '.format(str(float(progress / lenart)), str(progress), (1 / (end - start)))) #lets us see how much time is 
end2 = time.time()

print('waktu',end2-start2)

 0.2 percent, 1 position, 0.006133967805800569 per second 
gagal http://www.navytimes.com/story/news/nation-now/2015/12/31/ny-man-charged-islamic-state-new-years-eve-attack/78130922/
 0.4 percent, 2 position, 0.006040221865721991 per second 
gagal http://www.navytimes.com/story/military/2015/12/31/irans-president-orders-stepped-up-missile-production/78147134/
 0.6 percent, 3 position, 0.005994024473625889 per second 
gagal http://newsok.com/police-man-accused-of-road-rage-shooting-stayed-on-scene/article/feed/943163
 0.8 percent, 4 position, 0.005777073556630821 per second 
gagal http://www.13wmaz.com/story/news/local/macon/2015/12/31/two-wanted-for-robberies-in-four-states/78149932/
 1.0 percent, 5 position, 0.005596975928353311 per second 
waktu 18.09385848045349


In [ ]:
print(type(aaa))

In [22]:
len(data_gdelt)

1392383

# Write hasil ke json

In [24]:
#Tidak dijalankan jika data.json sudah ada
with open('data.json', 'w') as fp:
    json.dump(res, fp)

# Transform Count Vect to Tfidf

### read json file

In [25]:
with open('data.json', 'r') as fp:
    data_counter = json.load(fp)

### add event ID and bag of words to different array

In [26]:
eventID = []
DictVector = []

In [27]:
for k,v in data_counter.items():
    eventID.append(k)
    DictVector.append(v)

### vectorize bag of words from all news

In [28]:
vec = DictVectorizer()
DictArray = vec.fit_transform(DictVector).toarray()

### transform to TFIDF

In [29]:
tfidf_transformer = TfidfTransformer(use_idf=False)
tfidf_transform=tfidf_transformer.transform(DictArray, copy = True)
print(tfidf_transform)
print(DictArray)
print(tfidf_transform.todense())

  (0, 0)	0.017039954414774926
  (0, 1)	0.017039954414774926
  (0, 2)	0.017039954414774926
  (0, 3)	0.03407990882954985
  (0, 4)	0.03407990882954985
  (0, 5)	0.03407990882954985
  (0, 6)	0.017039954414774926
  (0, 7)	0.017039954414774926
  (0, 8)	0.017039954414774926
  (0, 9)	0.05111986324432477
  (0, 10)	0.017039954414774926
  (0, 11)	0.03407990882954985
  (0, 12)	0.017039954414774926
  (0, 13)	0.017039954414774926
  (0, 14)	0.05111986324432477
  (0, 15)	0.10223972648864954
  (0, 16)	0.017039954414774926
  (0, 17)	0.017039954414774926
  (0, 18)	0.08519977207387462
  (0, 19)	0.017039954414774926
  (0, 20)	0.03407990882954985
  (0, 21)	0.03407990882954985
  (0, 22)	0.017039954414774926
  (0, 23)	0.017039954414774926
  (0, 24)	0.017039954414774926
  :	:
  (0, 375)	0.017039954414774926
  (0, 376)	0.017039954414774926
  (0, 377)	0.017039954414774926
  (0, 378)	0.17039954414774924
  (0, 379)	0.017039954414774926
  (0, 380)	0.017039954414774926
  (0, 381)	0.017039954414774926
  (0, 382)	0.034

### header bag of words

In [30]:
vocabulary = vec.vocabulary_

In [31]:
# sort vocabulary by value (for header)
header = sorted(vocabulary.items(), key=operator.itemgetter(1))
header

[('abandon', 0),
 ('abduct', 1),
 ('abl', 2),
 ('accord', 3),
 ('accus', 4),
 ('act', 5),
 ('activ', 6),
 ('adam', 7),
 ('advanc', 8),
 ('affidavit', 9),
 ('afternoon', 10),
 ('agent', 11),
 ('agreement', 12),
 ('alert', 13),
 ('alleg', 14),
 ('allegedli', 15),
 ('allegi', 16),
 ('alreadi', 17),
 ('also', 18),
 ('although', 19),
 ('america', 20),
 ('american', 21),
 ('ammonia', 22),
 ('andrew', 23),
 ('angi', 24),
 ('ani', 25),
 ('anoth', 26),
 ('answer', 27),
 ('appar', 28),
 ('appear', 29),
 ('approxim', 30),
 ('arab', 31),
 ('arm', 32),
 ('arrest', 33),
 ('assault', 34),
 ('assist', 35),
 ('attack', 36),
 ('attempt', 37),
 ('attorney', 38),
 ('author', 39),
 ('avail', 40),
 ('avenu', 41),
 ('back', 42),
 ('bar', 43),
 ('becom', 44),
 ('befor', 45),
 ('beyond', 46),
 ('bloodi', 47),
 ('bomb', 48),
 ('bought', 49),
 ('brian', 50),
 ('brother', 51),
 ('build', 52),
 ('cabl', 53),
 ('caliph', 54),
 ('call', 55),
 ('cancel', 56),
 ('car', 57),
 ('carlin', 58),
 ('case', 59),
 ('caus', 60

In [32]:
header_tfidf=[]
for k,v in header:
    header_tfidf.append(k)

## 

In [34]:
matriks_tfidf = tfidf_transform.todense()
df = pd.DataFrame(matriks_tfidf)
df.to_csv("tfidf.csv", index=False, header=header_tfidf, mode='w')